<a href="https://colab.research.google.com/github/hydrationn/Machine-DeepLearning/blob/main/DeepLearning/lab/%EA%B3%BC%EC%A0%9C3_202101109_%E1%84%87%E1%85%A1%E1%86%A8%E1%84%89%E1%85%AE%E1%84%92%E1%85%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import package

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
# GPU/CPU setting

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

cpu


In [3]:
# import dataset

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 98778825.33it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 64952114.65it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30325275.68it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 2435506.11it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
# Batch generation

batch_size = 100

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Model

class CustomMLP(nn.Module):
    def __init__(self):
        super(CustomMLP, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the image
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dp1(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dp2(x)
        x = self.fc3(x)
        return x

In [6]:
# Parameter

num_epochs = 20

In [8]:
# Model definition

model = CustomMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

In [12]:
# Train

# Model Train
loss_list = []
for epoch_num in range(num_epochs):
  average_cost = 0
  model.train()

  for batch_idx, (x_data, y_label) in enumerate(train_loader):
    num_of_mini_batch = len(train_loader)

    x_data = x_data.reshape(-1, 28*28)
    input_image = x_data.to(device)
    label = y_label.to(device)
    optimizer.zero_grad()
    y_predict = model(input_image)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()

    average_cost = average_cost + (loss.item()/num_of_mini_batch)
    loss_list.append(loss)

  print("Epoch {} Loss {:.5f}".format((epoch_num+1), average_cost))

Epoch 1 Loss 0.04025
Epoch 2 Loss 0.03775
Epoch 3 Loss 0.03823
Epoch 4 Loss 0.03430
Epoch 5 Loss 0.03447
Epoch 6 Loss 0.03473
Epoch 7 Loss 0.03111
Epoch 8 Loss 0.03142
Epoch 9 Loss 0.03267
Epoch 10 Loss 0.03038
Epoch 11 Loss 0.02942
Epoch 12 Loss 0.03047
Epoch 13 Loss 0.02982
Epoch 14 Loss 0.02786
Epoch 15 Loss 0.02927
Epoch 16 Loss 0.02786
Epoch 17 Loss 0.02487
Epoch 18 Loss 0.02592
Epoch 19 Loss 0.02517
Epoch 20 Loss 0.02578


In [13]:
# Validation

with torch.no_grad():
  num_total_data = 0
  correct = 0
  model.eval()
  for batch_idx, (images, labels) in enumerate(test_loader):

    images = images.reshape(-1, 28*28)
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    outputs_softmax = F.softmax(outputs, dim=1)
    predicted = torch.argmax(outputs_softmax, dim=1)

    num_total_data += len(images)
    answer = sum(labels==predicted).item()
    correct += answer

print("Model accuracy {:.5f}%".format((correct/num_total_data)*100))

Model accuracy 98.58000%
